In [1]:
# System libraries
import os
import chromadb
import openai

# GenAI
from langchain.document_loaders import PyPDFLoader 
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from dotenv import load_dotenv

In [2]:
# const
CHROMA_PATH = "../chroma"
DATA_PATH = "../data/Student-Cookbook.pdf"

In [3]:
# Load document
loader = PyPDFLoader(DATA_PATH)
documents= loader.load()

Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 169 0 (offset 0)


In [4]:
# Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function =len,
    is_separator_regex=False,
)
chunks = text_splitter.split_documents(documents)

In [5]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY") 
openai.api_key = api_key

In [6]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [7]:
persist_directory = "../chroma_db"

db = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory)


In [8]:
user_question = "I need a quick lunch recipe" # User question
retrieved_docs = db.similarity_search(user_question, k=10) # k is the number of documents to retrieve

In [9]:
# Display top results
for i, doc in enumerate(retrieved_docs[:3]): # Display top 3 results
    print(f"Document {i+1}:\n{doc.page_content[36:1000]}") # Display content

Document 1:

Document 2:
 are a great lunch staple and are very simple and easy to make. Getting bored of 
plain old peanut butter and jelly? Here are some ideas from USU dietetic students to try to spice 
up your everyday sandwiches. 
 
Breads: 
Bread 
Tortilla 
Pita bread 
Naan bread (or other flat bread) 
Roll 
Bun 
French bread or other artisan breads 
English muffins 
Ciabatta  
 
Spread: 
Fancy mustard 
Hummus 
Pesto 
Cream cheese (plain or herbed) 
Mayonnaise (plain or flavored) 
Ranch or other salad dressings 
Salsa 
Guacamole 
Mashed beans 
 
Protein: 
Deli meat 
Grilled chicken 
Cooked egg 
Sausage 
Roasted turkey or other meat 
Tuna 
Cheese 
Beans 
Tofu 
Veggie Burgers 
Egg salad (or chicken salad) 
Lentils 
 
Vegetables: 
Lettuce 
Spinach
Document 3:
ches 
The cook time for this recipe is just 30 minutes, but it does take some time to prepare the dough. 
All you have to do is leave it out all day to rise and when you come home from class to make 
dinner, your dough is ready

In [10]:
def _get_document_prompt(docs):
    prompt = "\n"
    for doc in docs:
        prompt += "\nContent:\n"
        prompt += doc.page_content + "\n\n"
    return prompt

In [11]:
# Generate a formatted context from the retrieved documents
formatted_context = _get_document_prompt(retrieved_docs)

In [12]:
prompt = f"""
## SYSTEM ROLE
You are a helpful and kind chatbot designed to assist with questions related with cooking.
Your answers must be based exclusively on provided content from books provided. 

## USER QUESTION
The user has asked: 
"{user_question}"

## CONTEXT
Here is the relevant content from the technical books:  
'''
{formatted_context}
'''

## GUIDELINES
1. **Accuracy**:  
   - Only use the content in the `CONTEXT` section to answer.  
   - If you don't know the answer, just say: Sorry, I cannot help you with that.

2. **Transparency**:  
   - Reference the book's name and page numbers when providing information.  
   - Do not speculate or provide opinions.  

3. **Clarity**:  
   - Use simple and concise language.  
   - Format your response in Markdown for readability.  

## TASK
1. Answer the user's question **directly** if possible.  
2. When explaining a recipe, use bullet points for every ingredient and add the steps
3. When you don't know the answer don't respond with a title format and don't add the source
4. Provide the response in the following format:

## RESPONSE FORMAT

## [Brief Title of the Answer]
[Answer in simple, ordered, clear text.]

**Source**:  
• [Book Title], Page(s): [...]

"""
print("Prompt constructed.")

Prompt constructed.


In [13]:
# Set up GPT client and parameters
client = openai.OpenAI()
model_params = {
    'model': 'gpt-4o',
    'temperature': 0.7,  # Increase creativity
    'max_tokens': 4000,  # Allow for longer responses
    'top_p': 0.9,        # Use nucleus sampling
    'frequency_penalty': 0.5,  # Reduce repetition
    'presence_penalty': 0.6    # Encourage new topics
}

In [14]:
messages = [{'role': 'user', 'content': prompt}]
completion = client.chat.completions.create(messages=messages, **model_params, timeout=120)

In [15]:
answer = completion.choices[0].message.content
print(answer)

## Quick Lunch Recipe: Nut Butter and Banana Open Sandwich

Here's a simple and quick recipe for a delicious nut butter and banana open sandwich:

### Ingredients:
- 1 large banana
- 2 tablespoons of almond butter or peanut butter
- 1 tablespoon honey or maple syrup
- 1 tablespoon tahini (optional)
- Toppings of your choice: walnuts, raisins, almond flakes, sesame seeds, chia seeds

### Instructions:
1. Slice the banana lengthwise.
2. Mix the nut butter with tahini and sweetener of your choice.
3. Layer the mixture on the banana slices.
4. Add your desired toppings.

This is an easy lunch option and can also serve as a great afternoon snack or pre/post-workout snack.

**Source**:  
• [Book Title], Page(s): 23
